In [1]:
# https://python.langchain.com/v0.1/docs/expression_language/primitives/passthrough/#retrieval-example
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # type: ignore
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0) # type: ignore #,convert_system_message_to_human=True,temperature=0)



vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=gemini_embeddings
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough(passed=RunnablePassthrough())}
    | prompt
    | chat
    | StrOutputParser()
)

result = retrieval_chain.invoke("where did harrison work?")

In [2]:
print(type(result))

<class 'str'>


In [3]:
print(result)

Harrison worked at Kensho. 



In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed2=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

result = runnable.invoke({"num": 1})

In [12]:
print(type(result))
print(result.keys())

<class 'dict'>
dict_keys(['passed2', 'modified'])


In [4]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model  = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [5]:
result = chain.invoke({"foo": "bar", "bar": "gah"})
print(type(result))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
print(result.content)

3 + 9 = 12 



In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

model  = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)


chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

lang = "japanese"
result = chain2.invoke({"person": "obama", "language": lang})

In [13]:
print(type(result))
print(result)

<class 'str'>
バラク・オバマが生まれたホノルルは、**アメリカ合衆国**にあります。 



In [34]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

results = {}

runnable1 = RunnableParallel(
    color=RunnablePassthrough(),

    
)

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | runnable1| {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [35]:
result = question_generator.invoke("sharp")

In [38]:
print(type(result))
print(result)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[HumanMessage(content='What is the color of Blueberry \n and the flag of Dominica \n?')]
